In [ ]:
import os

from joblib import Parallel, delayed
import numpy as np
import pandas as pd
import tensorflow._api.v2.compat.v1 as tf
import sklearn
from sklearn.model_selection import GroupKFold, LeaveOneGroupOut
import matplotlib.pyplot as plt
plt.style.use('seaborn-poster')
from scipy import signal

from models import deeper_fcn as architecture
from scipy.fft import fft, fftfreq
import algorithms.heartrate as hr
import algorithms.movement_detection as md
import algorithms.common as cm
import algorithms.segmenter as sg
import utils
from algorithms.FFT import FFT as FFT
from datetime import timedelta

In [ ]:
# tensorflow settings
tf.logging.set_verbosity(tf.logging.ERROR)
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
tf.keras.backend.set_session(tf.Session(config=config))

In [ ]:
# load data
# x_data_train, y_data_train, groups_train = ...
# dummy:
#data_folder = 'data'

train_size = 774
n_groups = 28
data = np.genfromtxt('C:/Users/dario/Desktop/pr1/1000x400/rawprova1/raw_2022-03-14_bcg.csv', delimiter=';')
# print the shape of the data
#print(data.shape)

movsign=md.detect_movements(data[:512],50)

normalizzato=sg.renormalize_signal(data[:400],50)

filtered=cm.filter_bandpass(normalizzato,50,2,10)
filtered2=cm.filter_bandpass(data[:400],50,2,10)

print(movsign.shape)
plt.plot(normalizzato)
plt.plot(filtered2, label='BCG Filtrato')
plt.legend()
'''
plt.plot(movsign, label='movimento')
plt.legend()
plt.xlabel("Tempo[s]")
plt.ylabel("Ampiezza")
plt.show()
'''
hearths=hr.get_heartrate_pipe(filtered)
indicesijk=sg.find_ijk(filtered,50,[-1,-1,1])
print(indicesijk)
'''

#FOURIER TRANSFORM (DFT)

# Number of sample points
N = 500
# sample spacing
T = 1.0 / 50.0
x = np.linspace(0.0, N*T, N, endpoint=False)
y = filtered
yf = fft(y)
xf = fftfreq(N, T)[:N//2]
plt.stem(xf, 2.0/N * np.abs(yf[0:N//2]))
plt.grid()
plt.xlabel("Frequenza[Hz]")
plt.ylabel("Ampiezza")
plt.show()
'''
data = data[:len(data)-len(data)%400]
# reshape the data to be 2D array with 400 columns and 5 rows   (400, 5)

data = data.reshape( int(len(data)/400), 400)
# print the shape of the data
print(data.shape)

x_data_train = np.expand_dims(data, axis=2)

#y_train section
data = np.genfromtxt('C:/Users/dario/Desktop/pr1/1000x400/gold/gold.csv', delimiter=',')
print(len(data))
y_data_train = np.mean(data.reshape(int(len(data)/16), 16), axis=1)
y_data_train2=signal.resample(y_data_train,round(len(y_data_train)/2))


# compute average each 8 elements in the array
data = data[:len(data)-len(data)%16]
y_data_train = np.mean(data.reshape(int(len(data)/16), 16), axis=1)
y_data_train2=signal.resample(y_data_train,round(len(y_data_train)/2))
y_data_train = y_data_train[:774]
y= np.arange(0,400,1)

xmax = y[np.argmax(y_data_train2)]
ymax = y_data_train2.max()


#xposmin = np.where(y_data_train2==min)
#xmin = y[xposmin]
print(xmax)
print(ymax)
#y_timedelta = [str(timedelta(seconds=int(s))) for s in y]
#plt.plot(max,'o')
#plt.plot(min,'o')

#plt.plot(y_data_train[:400], label='gold')
#plt.xticks(y_data_train2[:400],y_timedelta)
plt.xlabel("Numero campioni ")
plt.ylabel("Magnitudo")
#plt.plot(y_data_train2[:100], label='y_train2')
#plt.plot(xmax,ymax,'X')
#plt.plot(xmin,ymin,'X')
print("zeros",len(y_data_train) - np.count_nonzero(y_data_train))
print(y_data_train.shape)
# print first 2 rows of the data
groups_train = np.sort(np.random.randint(n_groups, size=train_size))

print(x_data_train.shape, y_data_train.shape, groups_train.shape)

# print(x_data_train[0,:50])
# plot x_data_train

#plt.plot(x_data_train[:400], label='x_data_train')
# plt.plot(y_data_train[:50], label='y_data_train')
#plt.legend()

plt.show()



In [89]:
enlarge = 1
model_params = dict(metrics=["mae", "mape"], enlarge=enlarge)
fit_params = dict(epochs=30, verbose=2)  # set epochs between 30 and 75

modelname = (architecture.__name__ + "-x{}".format(enlarge))
modelpath = os.path.join("output", modelname)
os.makedirs(os.path.join(modelpath, "final"), exist_ok=True)
print(modelpath)

# write model architecture to JSON file
model = architecture.create(**model_params)
with open(os.path.join(modelpath, "model.json"), "w") as fp:
    fp.write(model.to_json())

output\models.deeper_fcn-x1


In [15]:
# single process for parallel training
def process_split(xt, yt, i, fit_params):
    # set allow_growth in subprocess
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    tf.keras.backend.set_session(tf.Session(config=config))
    
    csv_path = os.path.join(modelpath, "logs-{:02d}.csv".format(i))
    weights_path = os.path.join(modelpath, "weights-{:02d}.h5".format(i))
    fit_params.update(callbacks=[tf.keras.callbacks.CSVLogger(csv_path)])
    
    model = architecture.create(**model_params)
    r = model.fit(xt, yt, **fit_params)
    
    model.save_weights(weights_path)
    tf.keras.backend.clear_session()

    return r.history

In [16]:
# split training data with LeaveOneGroupOut cross validation
splitter = utils.get_group_splitter(n_groups, groups_train)

In [17]:
rs = Parallel(n_jobs=4, verbose=10)(
    delayed(process_split)(x_data_train[t_inds], y_data_train[t_inds],
                           i, fit_params)
    for i, (t_inds, v_inds) in enumerate(splitter)
)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:  3.5min
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:  4.3min
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:  5.6min
[Parallel(n_jobs=4)]: Done  24 out of  28 | elapsed:  7.2min remaining:  1.2min
[Parallel(n_jobs=4)]: Done  28 out of  28 | elapsed:  7.8min finished


In [18]:
model = utils.get_model_from_json(modelpath, "model.json")

# calculate MAPE and MAE for each left-out patient
splitter = LeaveOneGroupOut().split(x_data_train, y_data_train, groups=groups_train)
results = []
for i, (t_inds, v_inds) in enumerate(splitter):
    model.load_weights(os.path.join(modelpath, "weights-{:02d}.h5".format(i)))
    y_pred = model.predict(x_data_train[v_inds])
    y_true = y_data_train[v_inds]
    results.append((hr.hr_mape(y_true, y_pred), hr.hr_mae(y_true, y_pred)))
results = np.array(results)
display(results)

1/1 [==============================] - 0s 23ms/step


array([[32.90328934, 15.26580528],
       [29.82754639, 15.69148838],
       [34.90346695, 16.09085478],
       [43.2275186 , 19.06615125],
       [35.14385714, 16.22618612],
       [45.56579489, 19.60356421],
       [36.63916036, 15.99614742],
       [36.56766887, 16.5581481 ],
       [34.8638524 , 16.02547191],
       [36.65380186, 15.74567611],
       [36.40122055, 17.00071504],
       [33.99871409, 15.78173564],
       [40.43487234, 18.30919702],
       [35.72014228, 16.73106031],
       [44.9053473 , 20.22191915],
       [41.59316341, 18.27210377],
       [37.30987627, 16.89937582],
       [39.32880385, 18.84948001],
       [30.61743309, 15.54984255],
       [36.90251601, 16.98108338],
       [42.75683965, 18.55752985],
       [56.42545361, 20.06551266],
       [41.37010443, 17.78627777],
       [36.37237064, 16.59767401],
       [39.88016573, 16.98564293],
       [40.16471091, 18.05527265],
       [41.45487228, 19.83067557],
       [37.83879318, 17.85752449]])

In [20]:
# train one model on entire training set

model = architecture.create(**model_params)
r = model.fit(x_data_train, y_data_train, **fit_params)
model.save_weights(os.path.join(modelpath, "final", "weights-00.h5"))
tf.keras.backend.clear_session()

Epoch 1/30
25/25 - 3s - loss: 963.9180 - mae: 24.5322 - mape: 59.3805 - 3s/epoch - 131ms/step
Epoch 2/30
25/25 - 1s - loss: 474.5211 - mae: 18.1598 - mape: 42.4072 - 573ms/epoch - 23ms/step
Epoch 3/30
25/25 - 1s - loss: 410.3180 - mae: 16.7055 - mape: 37.2297 - 626ms/epoch - 25ms/step
Epoch 4/30
25/25 - 1s - loss: 393.6518 - mae: 17.0193 - mape: 40.1097 - 562ms/epoch - 22ms/step
Epoch 5/30
25/25 - 1s - loss: 354.0025 - mae: 15.4270 - mape: 34.2894 - 592ms/epoch - 24ms/step
Epoch 6/30
25/25 - 1s - loss: 352.4646 - mae: 15.9062 - mape: 35.7734 - 549ms/epoch - 22ms/step
Epoch 7/30
25/25 - 1s - loss: 327.7592 - mae: 15.2600 - mape: 34.1663 - 547ms/epoch - 22ms/step
Epoch 8/30
25/25 - 1s - loss: 319.8145 - mae: 14.9325 - mape: 33.3862 - 549ms/epoch - 22ms/step
Epoch 9/30
25/25 - 1s - loss: 308.5597 - mae: 14.6626 - mape: 32.8503 - 589ms/epoch - 24ms/step
Epoch 10/30
25/25 - 1s - loss: 304.6606 - mae: 14.3957 - mape: 32.1695 - 590ms/epoch - 24ms/step
Epoch 11/30
25/25 - 1s - loss: 296.1560 -